## Extraccion de los outputs, Version local

#### Funciona con archivos que terminan en .out generados por ORCA
   Por defecto solo va a seleccionar los datos que posean un wavelegth superior a 200 nm
#### Este notebook esta pensado para ser usado en un computador local, no para correr en ```google collab.```

In [ ]:
import os
import pandas as pd
import numpy as np
import re
from IPython.display import HTML
pd.set_option.max_columns_width = None
pd.options.display.max_rows = None

In [ ]:
minimo_espectro = float(input("escribe el minimo del espectro de la fortaleza del oscilador (ej: 0.01):"))
maximo_espectro = float(input("escribe el maximo del espectro de la fortaleza del oscilador (ej: 1):"))
minimo_longitud_onda = 200
path = "archivos" #modificar aca con el nombre de la carpetan que crearon dentro del directorio root, tiene que estar entre comillas

In [ ]:
#funciones necesarias para el programa. NO MODIFICAR
#directorio = os.path.join("aaaaaa", "tddft")
directorio = os.path.join("aaaaaa/tddft")
patron = r"au\s*([\d.]+)"
patron_transicion = r'\d+\.\d+'
patron_orbital = r'[-+]?\d*\.\d+|\d+'
def espacios_state(num):
    if len(str(num)) == 1:
        return  ' ' + str(num)
    elif len(str(num)) == 2:
        return str(num)
    
def string_transicion(num: int):
    if num > 0:
        return "+"+str(num)
    if num < 0:
        return "-"+str(abs(num))
    if num == 0:
        return ""
    
def convertir_transicion(string: str, homo: int, lumo: int):
    string = string.replace("a", "")
    string = string.split("-")
    diferencia_homo = int(string[0]) - homo
    diferencia_lumo = int(string[1]) - lumo
    diferencia_homo = string_transicion(diferencia_homo)
    diferencia_lumo = string_transicion(diferencia_lumo)
    return "H"+diferencia_homo+"→L"+diferencia_lumo

def encontrar_keys(dicc: dict):
    llave = []
    valor = None
    for key, value in dicc.items():
        if valor is not None and value != valor:
            llave.append(key)
        valor = value
    llave = int(llave[0])
    return llave

def log(mensaje: str):
        print("|" + (" "+mensaje+" ").center(50, "-") + "|")

In [ ]:
#indica los archivos que se leeran en el path dado
def obtener_archivos_out(extension):
    file_list = []
    for archivo in os.listdir(directorio):
        if archivo.endswith(extension):
            file_list.append(archivo)
    return file_list

archivos = obtener_archivos_out('.out')

log("Lista de archivos out")
for filename in archivos:
    print(filename)

log(f"Numero total de archivos: {len(archivos)}")

In [ ]:
%%time
%%capture captured_output
for filename in archivos:
    tabla = pd.DataFrame(columns=["State", "E(eV)", "λ(nm)", "f", "Assignment", "MOs Transition", "cts (%)"])
    columnas = ["State", "Energy (cm-1)", "Wavelength (nm)", "fosc", "T2 (au**2)", "TX (au)", "TY (au)", "TZ (au)"]
    log(filename.split(".out")[0].split("td_")[1])
    df = pd.DataFrame(columns=columnas)
    with open(directorio + "/"  + filename, "rt") as file:
        for key, linea in enumerate(file):
            if "ABSORPTION SPECTRUM VIA TRANSITION ELECTRIC DIPOLE MOMENTS" in linea:
                #print(f"Linea en donde estan los datos sobre el espectro de absorcion: {key+1}")
                while True:
                    datum = file.readline()
                    if datum.strip() == "":
                        df.drop(["T2 (au**2)", "TX (au)", "TY (au)", "TZ (au)"], axis=1, inplace=True)
                        df["State"] = df["State"].astype(np.uint8)
                        df["Wavelength (nm)"] = df["Wavelength (nm)"].astype(np.float16)
                        df[["Energy (cm-1)", "fosc"]] = df[["Energy (cm-1)", "fosc"]].astype(np.float32)
                        break
                    numeros = [numero for numero in datum.split()]
                    if not numeros[0].isnumeric():
                        continue
                    df.loc[len(df.index)] = numeros
                df = df.loc[(df["fosc"] >= minimo_espectro) & (df["fosc"] <= maximo_espectro) & (df["Wavelength (nm)"] >  minimo_longitud_onda)]
                df.reset_index(drop=True, inplace=True)
                break
        #calcular los estados
        file.seek(0)
        for key, line in enumerate(file):
            if "NO" in line and "OCC" in line and "E(Eh)" in line and "E(eV)" in line:
                orbitales = {}
                while True:
                    datum = file.readline()
                    if datum[0] == "-":
                        break
                    numeros = re.findall(patron_orbital, datum)
                    orbitales[numeros[0]] = numeros[1]
                lumo = encontrar_keys(orbitales)
                homo = lumo - 1
        #agrega los datos en una tabla
        file.seek(0)
        tabla[["State", "λ(nm)", "f"]] = df[["State", "Wavelength (nm)", "fosc"]]
        tabla["Assignment"] = "π→π*"
        tabla["Assignment"] = tabla["Assignment"].astype("category")
        for key, linea in enumerate(file):
            for x in range(len(df)):
                estado = df['State'].iloc[x]
                if str("STATE " + espacios_state(estado)) in linea:
                    energia = float(re.search(patron, linea).group(1))
                    tabla.loc[tabla.State == int(estado), "E(eV)"] = energia
                    estados = {}
                    while True:
                        datum = file.readline()
                        if datum.strip() == "":
                            transicion = max(estados, key=estados.get)
                            max_cts = estados[transicion]
                            tabla.loc[tabla.State == int(estado), "MOs Transition"] = convertir_transicion(transicion, homo, lumo)
                            tabla.loc[tabla.State == int(estado), "cts (%)"] = max_cts
                            break
                        estados[datum.split(":")[0].replace(' ', '').replace('>', '')] = float(re.search(patron_transicion, datum.split(":")[1]).group())
        tabla['λ(nm)'] = tabla['λ(nm)'].round(0).astype(int)
        tabla["E(eV)"] = tabla["E(eV)"].astype(float).round(2)
        tabla['f'] = tabla['f'].round(2)
        tabla['cts (%)'] = tabla['cts (%)'].astype(float).round(2) * 100
        tabla['cts (%)'] = tabla['cts (%)'].astype(int).map(lambda x: f"{x}%")
        tabla.drop(["State"], axis=1, inplace=True)
        tabla.set_index("E(eV)", inplace=True)
        tabla.columns.name = tabla.index.name
        tabla.index.name = None
        print(tabla)
log("PROCESO TERMINADO")
%%writefile output.txt
print(captured_output.stdout)

In [ ]:
with open("output.txt", "w", encoding="utf-8") as file:
    file.write(captured_output.stdout)
print("listo")